In [1]:
# Importing necessary libraries
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

# Defining the provided functions and parameters

# Define the weight function
def omega(t, u, x, c1, c2):
    sigma = math.sqrt(np.var(Y))
    h1 = (c1/math.sqrt(12))*(T ** (-(1/5)))
    h2 = c2*sigma*(T ** (-(1/5)))
    numerator = K_h1(u - t / T, h1) * np.prod(K_h2(x - Y[t-1], h2))
    denominator = np.sum([K_h1(u - s / T, h1) * np.prod(K_h2(x - Y[s-1], h2)) for s in range(1, T+1)])
    return numerator / denominator

def ecdf(u, x, v):
    weights = np.array([omega(t, u, x, c1, c2) for t in range(1, T+1)])
    indicator = (Y <= v).astype(int)
    return np.sum(weights * indicator)

# Define uniform kernels
def K_h1(z, h1):
    return 0.5 if abs(z/h1) <= 1 else 0

# Define Gaussian kernels
def K_h2(z, h2):
    return np.exp(-z**2 / (2 * h2**2))

# Parameters
T = 400  # Length of time series
d = 1  # Number of covariates
c1 = 5
c2 = 1.68
u_test = 0.5
x_test = np.array([0.5])
v_values = np.linspace(-2, 2, T)  # Placeholder values, will be updated in the loop

# Running the replication loop
W1_list = []

for rep in range(100):
    # Generate locally stationary data
    np.random.seed(rep)  # Use different seed for each replication
    Y = np.sin(np.linspace(0, 4 * np.pi, T)) + np.random.normal(0, 0.5, T)  # Time series with noise
    
    v_values = np.linspace(Y.min(), Y.max(), T)  # Update v_values based on new Y
    
    h1 = (c1/math.sqrt(12))*(T ** (-(1/5)))
    h2 = c2*math.sqrt(np.var(Y))*(T ** (-(1/5)))
    
    # Test the weight function for a given (u, x)
    weights = [omega(t, u_test, x_test, c1, c2) for t in range(1, T+1)]
    
    # Values for plotting
    F_values = [ecdf(u_test, x_test, v) for v in v_values]
    
    # True CDF
    data_sorted = np.sort(Y)
    True_CDF = np.arange(1, len(data_sorted) + 1) / len(data_sorted)
    
    # Compute the 1-Wasserstein distance
    W1 = wasserstein_distance(True_CDF, F_values)
    W1_list.append(W1)

# Computing mean 1-Wasserstein distance
mean_W1 = np.mean(W1_list)

print("1-Wasserstein Distance: ", W1_list)
print("Average 1-Wasserstein Distance: ", mean_W1)


1-Wasserstein Distance:  [0.21108458211379288, 0.20039327564444998, 0.19531465141159338, 0.1950218526601817, 0.17122013364316674, 0.20065031303102637, 0.21327363691551826, 0.19280971285740348, 0.2005806883800895, 0.18235089629088447, 0.20958511384688328, 0.1949145293191054, 0.1971280788162273, 0.203713083884847, 0.18748635659307142, 0.21688358694316395, 0.20611138167968868, 0.1852035875386072, 0.19871859568743183, 0.2196574737718175, 0.19794482758530907, 0.21415630466300478, 0.1859220018448342, 0.18262403948886774, 0.19146676989114358, 0.20098375071615462, 0.18489793228014853, 0.19569956235436176, 0.18704574611717895, 0.18953006715129445, 0.19441271623781836, 0.19637958690317076, 0.18899597655948286, 0.18737999194770527, 0.20424002723190157, 0.20161562130077115, 0.1964988952692604, 0.19770653579829583, 0.18165301621365865, 0.19216836085624256, 0.20622412347648467, 0.19255592121822498, 0.18020179058526625, 0.19539711735383516, 0.20618670835921632, 0.1966215642124325, 0.19480669917135834